# Ensemble learning

In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mlp
import seaborn as sns
import re, pip, conda

d:\CODE\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


---

- ### Boosting

- **Boosting PK Bagging**

||装袋法 Bagging|提升法 Boosting|
|-|-|-|
|弱评估器|**相互独立**，并行构建|**相互关联**，按顺序依次构建<br>先建弱分类器的预测效果影响后续模型的建立|
|建树前的抽样方式|样本有放回抽样<br>特征无放回抽样|样本有放回抽样<br>特征无放回抽样<br>先建弱分类器的预测效果可能影响抽样细节|
|集成的结果|回归平均<br>分类众数|每个算法**具有自己独特的规则**，一般来说：<br>(1) 表现为某种分数的加权平均<br>(2) 使用输出函数|
|目标|**降低方差**<br>提高模型整体的稳定性来提升泛化能力<br>本质是从“平均”这一数学行为中获利|**降低偏差**<br>提高模型整体的精确度来提升泛化能力<br>相信众多弱分类器叠加后可以等同于强 学习器|
|单个评估器容易<br>过拟合的时候|具有一定的抗过拟合能力|具有一定的抗过拟合能力|
|单个评估器的效力<br>比较弱的时候|可能失效|大概率会提升模型表现|
|代表算法|随机森林|梯度提升树，Adaboost|
|Weak evaluators|**Independent**, built in parallel|**Interrelated**, built in sequence<br>The prediction effect of the weak classifier built first affects the establishment of subsequent models|
|Sampling method before tree establishment|Sample sampling with replacement<br>Feature sampling without replacement|Sample sampling with replacement<br>Feature sampling without replacement<br>The prediction effect of building a weak classifier first may affect the sampling details|
|Integrated results|Regression average<br>Classification mode|Each algorithm**has its own unique rules**, generally speaking:<br>(1) It is expressed as a weighted average of a certain score<br>(2 ) using the output function |
|Goal|**Reduce variance**<br>Improve the overall stability of the model to improve generalization ability<br>The essence is to profit from the mathematical behavior of "average"|**Reduce bias**<br>Improve Improve the generalization ability by improving the overall accuracy of the model<br>I believe that the superposition of many weak classifiers can be equivalent to a strong learner|
|A single evaluator is prone to<br>over-fitting|Has a certain ability to resist over-fitting|Has a certain ability to resist over-fitting|
|The effectiveness of a single estimator<br>When it is weak|may fail|high probability will improve model performance|
|Representative Algorithm|Random Forest|Gradient Boosting Tree, Adaboost|

![RF2](https://pictes.oss-cn-beijing.aliyuncs.com/%E5%BE%AE%E8%AF%BE%20-%20sklearn/RFC/RF2.png)

In the Bagging algorithm represented by Random Forest, we build multiple parallel independent weak evaluators at once and let all evaluators operate in parallel. In Boosting integration algorithm, we build multiple weak evaluators (basically decision trees) one by one, and the next weak evaluator is built in a way that depends on the evaluation result of the previous weak evaluator, and finally the results of multiple weak evaluators are combined for the output, so that the weak evaluators in the Boosting algorithm are not only not independent from each other, but also strongly correlated with each other. Boosting algorithm also does not rely on the independence of weak classifiers to improve the results, which is a major difference between Boosting and Bagging. If the core difference between the different algorithms of Bagging is that they rely on different ways of achieving "independence" (randomness),** then the core difference between the different algorithms of Boosting is how the evaluation of the previous weak evaluator affects the creation of the next weak evaluator**.

Unlike the uniform regression-to-average and classification minority-to-majority outputs of the Bagging algorithms, the Boosting algorithms exhibit a great deal of variety in their resultant outputs. While the output of early Boosting algorithms was generally the output of the last weak evaluator, the outputs of contemporary Boosting algorithms consider all of the weak evaluators in the entire integrated model. **In general, each Boosting algorithm will customise the specific form of the integration output with its own unique rules**, but for most algorithms, the output of the integration algorithm tends to be a weighted average** of some kind of result about the weak evaluators, where solving for the weights is a very critical step in the field of boosting.

- **Basic Elements of Boosting Algorithms and Basic Processes**

- Loss function $L(x,y)$: a measure of the difference between the model's predictions and the true results.
- Weak evaluator $f(x)$: (typically) a decision tree, different boosting algorithms use different tree building processes.
- Comprehensive integration result $H(x)$: i.e., how exactly the integration algorithm outputs the integration result

**依据上一个弱评估器$f(x)_{t-1}$的结果，计算损失函数$L(x,y)$，
    <br>并使用$L(x,y)$自适应地影响下一个弱评估器$f(x)_t$的构建。<br>集成模型输出的结果，受到整体所有弱评估器$f(x)_0$ ~ $f(x)_T$的影响.**

- **boosting algorithm in sklearn**

**GBDT**（Gradient Boosting Decision Tree），**直方提升树**（Hist Gradient Boosting Tree）,**XGBoost**（Extreme Gradient Boosting Tree），轻量梯度提升树**LightGBM**（Light Gradiant Boosting Machine），以及离散提升树**CatBoost**（Categorial Boosting Tree）。

|Boosting算法|库|集成类|
|:--:|:--:|:--:|
|ADB分类|sklearn|AdaBoostClassifer|
|ADB回归|sklearn|AdaBoostRegressor|
|梯度提升树分类|sklearn|GradientBoostingClassifier|
|梯度提升树回归|sklearn|GradientBoostingRegressor|
|直方提升树分类|sklearn|HistGraidientBoostingClassifier|
|直方提升树回归|sklearn|HistGraidientBoostingRegressor|
|极限提升树|第三方库xgboost|xgboost.train()|
|轻量梯度提升树|第三方库lightgbm|lightgbm.train()|
|离散提升树|第三方库catboost|catboost.train()|

- ### AdaBoost (Adaptive Boosting)

1. For the first time, the result of the previous weak estimator can be adaptively affected in the subsequent modeling process<br>
2. In the Boosting algorithm, for the first time, an output method that considers all weak evaluator results is implemented<br>

As a pioneering algorithm, the construction process of AdaBoost is very simple: **First, establish a decision tree on all samples, and increase the sample weight of the incorrectly predicted samples in the data set based on the predicted results and loss function value of the decision tree. And let the weighted data set be used to train the next decision tree**. This process is equivalent to intentionally increasing the weight of "samples that are difficult to be classified correctly", while reducing the weight of "samples that are easy to be classified correctly", and directing the attention of the weak evaluator to be built subsequently to samples that are difficult to be classified correctly. on the sample.
![](https://skojiangdoc.oss-cn-beijing.aliyuncs.com/2021MachineLearning/Ensembles/Public/boostrap-fixed2.png)
|参数|参数含义|
|:-:|:-:|
|**base_estimator**|弱评估器|
|n_estimators|集成算法中弱评估器的数量|
|**learning_rate**|迭代中所使用的学习率|
|**algorithm**（分类器专属）|用于指定分类ADB中使用的具体实现方法|
|**loss**（回归器专属）|用于指定回归ADB中使用的损失函数|
|random_state|用于控制每次建树之前随机抽样过程的随机数种子|

In [2]:
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.ensemble import AdaBoostRegressor as ABR
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import DecisionTreeRegressor as DTR
from sklearn.datasets import load_digits

In [3]:
#for classification
data_c = load_digits()
X_c = data_c.data
y_c = data_c.target

In [4]:
X_c.shape

(1797, 64)

In [5]:
X_c

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [6]:
np.unique(y_c)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [7]:
#for regression
data_r = pd.read_csv(r"D:\Practice\Machine Learning\datasets\House Price\train_encode.csv",index_col=0)
X_g = data_r.iloc[:,:-1]
y_g = data_r.iloc[:,-1]

In [8]:
X_g.shape

(1460, 80)

In [9]:
X_g.head()

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,半开放式门廊面积,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0


In [10]:
clf = ABC(n_estimators=3).fit(X_c, y_c)
reg = ABR(n_estimators=3).fit(X_g, y_g)

In [11]:
clf.base_estimator_

d:\CODE\Lib\site-packages\sklearn\utils\deprecation.py:101: FutureWarning: Attribute `base_estimator_` was deprecated in version 1.2 and will be removed in 1.4. Use `estimator_` instead.
  warnings.warn(msg, category=FutureWarning)


DecisionTreeClassifier(max_depth=1)

In [12]:
reg.base_estimator_

DecisionTreeRegressor(max_depth=3)

In [13]:
reg.estimators_

[DecisionTreeRegressor(max_depth=3, random_state=475939812),
 DecisionTreeRegressor(max_depth=3, random_state=2109521539),
 DecisionTreeRegressor(max_depth=3, random_state=607322565)]

- `learning_rate`

In the Boosting ensemble method, the output $H(x)$ of the ensemble algorithm is often the weighted average of the output results of multiple weak evaluators. However, $H(x)$ is not uniformly weighted and solved after all trees are built, but is continuously calculated with iterations as the algorithm gradually builds trees. For example, for sample $x_i$, there are $T$ trees in the ensemble algorithm (that is, the value of parameter `n_estimators`). Now the $t$th weak evaluator is being established, then the $t$th weak evaluator is being established. The result of $x_i$ on the device can be expressed as $f_t(x_i)$. Assuming that the entire Boosting algorithm outputs a result of sample $x_i$ as $H(x_i)$, the result can generally be expressed as the weighted sum of all weak evaluator results in the process of t=1~t=T:
$$H(x_i) = \sum_{t=1}^T\phi_tf_t(x_i)$$

Among them, $\phi_t$ is the weight of the t-th tree. For the $t$th iteration, there is:

$$H_t(x_i) = H_{t-1}(x_i) + \phi_tf_t(x_i)$$

In this general process, each time the decision tree built in this round is added to the previous tree building results, the parameter $\color{red}\eta$ can be added in front of the weight $\phi$, which is expressed as the tth tree added The learning rate of the overall integration algorithm, benchmarked against the parameter `learning_rate`.

$$H_t(x_i) = H_{t-1}(x_i) + \boldsymbol{\color{red}\eta} \phi_tf_t(x_i)$$

This learning rate parameter controls the growth rate of $H(x_i)$ during the Boosting integration process and is a very critical parameter. When the learning rate is large, $H(x_i)$ grows faster and we need fewer n_estimators. When the learning rate is small, $H(x_i)$ grows slower and we need more n_estimators. , so the boosting algorithm often needs to make a trade-off between n_estimators and learning_rate (take the XGBoost algorithm as an example).

![](https://pictes.oss-cn-beijing.aliyuncs.com/%E5%BE%AE%E8%AF%BE%20-%20sklearn/week%2011%20XGBoost/eta.PNG)

> - `algorithm`
>
> **The base_estimators that can be input by default in sklearn also need to be weak estimators that can output predicted probabilities. In actual prediction, the $H(x)$ output by AdaBoost is also aimed at the probability of a certain category**.
>
> In the classifier, although we are allowed to choose the algorithm, we are not allowed to choose the loss function used by the algorithm. This is because SAMME and SAMME.R use the same loss function: binary exponential loss (Exponential Loss Function) and multiple classification Multi-class Exponential loss function.

Two-class exponential loss (Exponential Loss Function) and multi-class exponential loss (Multi-class Exponential loss function).

**Two classification index loss**——
$$L(H(x),y) = e^{-yH^*(x)}$$
Among them, y is the real classification, and $H^*(x)$ is the vector converted from the probability result $H(x)$ output by the ensemble algorithm. The conversion rules are as follows:

$$H^*(x)=
\begin{cases}
1& if \ H(x)>0.5 \\
-1& if\ H(x) < 0.5
\end{cases}$$

In sklearn, since $H(x)$ is a probability value, it needs to be converted to $H^*(x)$. If in other algorithm libraries that implement AdaBoost, the output of $H(x)$ is directly the predicted category. Then the conversion process does not need to be performed.

**According to the special properties of exponential loss, the category value in the two-classification situation can only be -1 or 1**, so the value of $y$ can only be -1 or 1. When the algorithm predicts correctly, the sign of $yH^*(x)$ is positive, and the loss on the function $e^{-x}$ is very small. When the algorithm predicts incorrectly, the sign of $yH^*(x)$ is negative, and the loss on the function $e^{-x}$ is large. Binary classification exponential loss is the most classic loss function of AdaBoost. Its effectiveness in mathematical derivation and strong guidance in practice allow it to be used today.

![](https://www.tf.uni-kiel.de/matwis/amat/mw1_ge/kap_5/illustr/exponential1com.png)

**Multi-category index loss**——

$$
\begin{aligned}
L(H(x),y) &=exp \left( -\frac{1}{K}\boldsymbol{y^* · H^*(x)} \right) \\
& = exp \left( -\frac{1}{K}(y^{*1}H^{*1}(x)+y^{*2}H^{*2}(x) \ + \ ... + y^{*k}H^{*k}(x)) \right)
\end{aligned}
$$<br>
Among them, $K$ is the total number of categories. For example, in the case of four categories [0,1,2,3], $K=4$, $\boldsymbol{y^*}$ and $\boldsymbol{H^*( x)}$ are all vectors converted according to the specific situation of multi-classification and the actual output $H(x)$ of the integration algorithm, among which $y^{*1}$ and $H^{*1}(x)$ The superscript 1 indicates the current category.

In the two-classification algorithm, the algorithm will directly output the probability for one of the categories in the two-classification, because in the two-classification $P(Y=1) = 1 - P(Y=-1)$, so only Calculating the probability of a class can determine the predicted label. However, in a multi-classification algorithm, the algorithm must output probabilities for all possible value types, so that the prediction label corresponding to the maximum probability can be found. Therefore, in the ensemble algorithm, when we perform multi-class prediction, we will get the following table:

In [14]:
clf = DTC(max_depth=2).fit(X_c,y_c)

pd.DataFrame(clf.predict_proba(X_c)).iloc[:5,:]

,0,1,2,3,4,5,6,7,8,9
0,0.909574,0.000000,0.010638,0.000000,0.031915,0.031915,0.015957,0.000000,0.000000,0.000000
1,0.003781,0.131380,0.120038,0.157845,0.134216,0.011342,0.003781,0.163516,0.158790,0.115312
2,0.003781,0.131380,0.120038,0.157845,0.134216,0.011342,0.003781,0.163516,0.158790,0.115312
3,0.000000,0.092672,0.099138,0.032328,0.071121,0.312500,0.370690,0.012931,0.006466,0.002155
4,0.909574,0.000000,0.010638,0.000000,0.031915,0.031915,0.015957,0.000000,0.000000,0.000000


Each row corresponds to a sample, and each column corresponds to the probability that the predicted label of the sample is a certain category. The above table is the probability table obtained by 5 samples under 10 classifications, ** and among the 10 probabilities of each sample , the category corresponding to the maximum probability is the predicted category**. This conversion can be completed by the function argmax. argmax will take out the index corresponding to the maximum value, which happens to be the prediction label corresponding to the maximum probability.

> - `loss`

"linear"（线性）,"square"（平方）,"exponential"（指数）

**R2算法线性损失——**

$$L_i = \frac{|H(x_i) - y_i|}{D}$$

**R2算法平方损失——**

$$L_i = \frac{|H(x_i) - y_i|^2}{D^2}$$

**R2算法指数损失——**

$$L_i = 1 - exp \left( \frac{-|H(x_i) - y_i|}{D} \right)$$

|参数|参数含义|
|:-:|:-:|
|**base_estimator**|弱评估器|
|n_estimators|集成算法中弱评估器的数量|
|**learning_rate**|迭代中所使用的学习率|
|**algorithm**（分类器专属）|用于指定分类ADB中使用的具体实现方法|
|**loss**（回归器专属）|用于指定回归ADB中使用的损失函数|
|random_state|用于控制每次建树之前随机抽样过程的随机数种子|

> - 1) Initialize the weight $w_i$ of the original data set, where any $w_i = \frac{1}{M}$

> - Start looping, for t in 1,2,...T:

> - 2) In the existing data set $N$, there are $M$ samples with replacement sampling, forming a training set $N^t$. Every time a sample is drawn, the probability of any sample being drawn is $P_i^t = \frac{w_i}{\sum w_i}$. Obviously, **this probability is the current sample in the training set $N^t Weight in $**. When sampling from the initial weights, the probability $P_i^1 = \frac{1}{M}$, and when subsequent weights change, samples with greater weights will have a greater probability of being selected. <br><br>
> - 3) Build a regression tree $f^t$ on the training set $N^t$ according to the **CART tree** rules. The label fitted during training is the **real label**$y of the sample. ^t_i$. <br><br>
> - 4) Input all the samples on $N^t$ into $f^t$ for prediction, and get the prediction result $f^t(x_i)$, where i = 1,2,...M. <br><br>
> - 5) Calculate the loss function $L^t_i = L(f^t(x_i),y_i)$ on a single sample $i$. The calculation process is as follows:
>> Solve $D = sup|f^t(x_i) - y_i|, i = 1,2,...,N$<br><br>
>> Select one of linear/square or exponential loss functions to calculate $L^t_i$<br><br>
>> Linear loss: $L_i = \frac{|f^t(x_i) - y_i|}{D}$<br><br>
>> Square loss: $L_i = \frac{|f^t(x_i) - y_i|^2}{D^2}$<br><br>
>> Exponential loss: $L_i = 1 - exp \left( \frac{-|f^t(x_i) - y_i|}{D} \right)$<br><br>
>> According to the requirements of AdaBoost, the value range of all losses is between [0,1].
> - 6) Calculate the weighted average loss on the entire sample $\bar{L^t} = \sum_{i=1}^ML_i^tP_i^t$
>> Note that $P_i^t$ is equal to the weight of the sample at this time. Since $P_i^t = \frac{w_i}{\sum w_i}$, so $P_i^t$ must be in the range [0,1], and $\sum{P_i^t}, i=1,2, ...M$ must be 1. <br><br>
>> **When the sum of weights is 1, the weighted average will definitely be less than or equal to the maximum value of a single value (and greater than or equal to the minimum value of a single value), so the value range of the weighted average will not exceed the value range of the single average. **. Since the range of all losses is [0,1], the range of the weighted average $\bar{L^t}$ is also [0,1]. At the same time, since the maximum value of the loss is 1, and the maximum value of the weight $P_i^t$ must be far less than 1, the maximum value of the weighted average $\bar{L^t}$ is generally far less than 1 of. <br>

> - 7) Calculate and measure the confidence of the current integration algorithm $\beta^t$ based on the weighted average loss $\bar{L^t}$
>> $\beta^t = \frac{\bar{L^t}}{1-\bar{L^t} + \lambda}$, where $\lambda$ is a constant to prevent the denominator from being 0<br ><br>
>> It is not difficult to find that when the weighted average loss is very high, $\beta^t$ is very large, so the confidence is small. When the weighted average loss is very low, $\beta^t$ is very small, so the confidence is large. . The greater the confidence, the better the current prediction result of the ensemble algorithm. <br><br>
>> It is known that the theoretical value range of $\bar{L^t}$ is [0,1], so the theoretical value range of $\beta^t$ is [0,$+\infty$], so $\beta_t The closer the value of $ is to 0, the better. <br><br>
>>At the same time, we also know that the actual range of $\bar{L^t}$ is approximately between 0.2 and 0.3, so generally speaking, the actual range of $\beta^t$ is basically less than 1.
> - 8) 依据置信度评估$\beta_t$更新样本权重
>> $w_i = w_i\beta^{(1-L_i)}$<br><br>
>> 我们可以根据$L_i$的范围[0,1]，以及$\beta$的计算公式，绘制出横坐标为$L_i$，纵坐标为$\beta^{(1-L_i)}$的图像。不难发现，**单一样本的损失越大、$\beta^{(1-L_i)}$也会越大，因此该样本的权重会被更新得越大**。<br>
> - 9) 求解迭代过程中弱分类器$f^t$所需的权重
>> $\phi^t = log(\frac{1}{\beta^t})$\
>> 其中log的底数为e或者为2皆可。当$\beta$值越接近于0，说明损失越小、置信度越高，则$log(\frac{1}{\beta^t})$的值越大。所以，损失更小的树对应的权重更大，损失更大的树对应的权重更小。
> - 10) 求解出当前迭代$t$下集成算法的输出值：
>> $H^t(x_i) = H^{t-1}(x_i) + \eta \phi^t f^t(x_i)$

> 《Multi-class AdaBoost》& sklearn source code(https://github.com/scikit-learn/scikit-learn/blob/0d378913b/sklearn/ensemble/_weight_boosting.py#L913)。